# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
weather_df = pd.read_csv('cities.csv')
weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,coquimbo,-29.95,-71.34,58.73,77,0,11.50,CL,1658856534
1,1,dalen,52.70,6.76,67.05,62,98,11.01,NL,1658856535
2,2,komsomolskiy,40.43,71.72,90.32,31,12,6.91,UZ,1658856535
3,3,buraydah,26.33,43.98,104.45,11,89,8.23,SA,1658856535
4,4,kapaa,22.08,-159.32,79.12,80,0,18.41,US,1658856536
...,...,...,...,...,...,...,...,...,...,...
557,557,bouafle,6.99,-5.74,77.72,71,100,0.04,CI,1658856735
558,558,el rosario,24.26,-107.18,89.91,57,100,6.98,MX,1658856735
559,559,luyang,27.62,110.10,77.58,76,52,3.65,CN,1658856735
560,560,misratah,32.38,15.09,84.11,61,0,9.48,LY,1658856736


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
locations = weather_df[["Lat", "Lng"]].astype(float)
humid = weather_df["Humidity"].astype(float)

In [26]:
#plot heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid,
                                dissipating=False, max_intensity=100,
                                point_radius=1)
#Add layer
fig.add_layer(heat_layer)

#display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
#weather conditions = 70>max_temp<80, windspeed<10, cloudiness = 0
#filter out temps >=70
maxtemp_70 = weather_df['Max Temp'] >=70
clean_maxtemp_70 = weather_df[maxtemp_70]

#filter out temps <=80 
maxtemp_80 = clean_maxtemp_70['Max Temp'] <=80
clean_maxtemp_80 = clean_maxtemp_70[maxtemp_80]

#filter out windspeed<10
windspeed_10 = clean_maxtemp_80['Wind Speed'] <10
clean_windspeed_10 = clean_maxtemp_80[windspeed_10]
clean_windspeed_10

#filter out cloudiness = 0
cloudiness_0 = clean_windspeed_10['Cloudiness'] ==0
clean_weather_df =clean_windspeed_10[cloudiness_0]
clean_weather_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
145,145,castro,-24.79,-50.01,74.95,41,0,2.28,BR,1658856302
152,152,koungou,-12.73,45.20,74.91,78,0,9.22,YT,1658856584
202,202,santa maria,-29.68,-53.81,70.90,78,0,9.22,BR,1658856459
309,309,erzin,36.96,36.20,77.40,66,0,3.91,TR,1658856638
474,474,steinbach,50.17,8.57,79.00,36,0,9.22,DE,1658856704
533,533,kijang,35.24,129.21,76.73,88,0,7.29,KR,1658856726


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#List for holding hotels
hotel_df = pd.DataFrame(clean_weather_df)
hotel_df["Hotel Name"] = ""

params = {
    "radius": 5000,
    "types": "hotel",
    "key": g_key    
}
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params = params)
    
    response = response.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result...skipping.")
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
145,145,castro,-24.79,-50.01,74.95,41,0,2.28,BR,1658856302,Castro
152,152,koungou,-12.73,45.20,74.91,78,0,9.22,YT,1658856584,Mamoudzou
202,202,santa maria,-29.68,-53.81,70.90,78,0,9.22,BR,1658856459,Santa Maria
309,309,erzin,36.96,36.20,77.40,66,0,3.91,TR,1658856638,Erzin
474,474,steinbach,50.17,8.57,79.00,36,0,9.22,DE,1658856704,Frankfurt
533,533,kijang,35.24,129.21,76.73,88,0,7.29,KR,1658856726,Busan


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [16]:
# Convert hotel name to list
hotel_list = hotel_df["Hotel Name"].tolist()

# Add marker layer ontop of heat map
marker_locations = hotel_df[['Lat', 'Lng']]

# Create a marker_layer using the hotel list to fill the info box
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))